In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import IPython
import requests
import dotenv

from backend.location import Location, make_grid
from backend.gmaps import get_static_map

dotenv.load_dotenv()

In [ ]:
GMAPS_API_KEY = os.environ["GMAPS_API_KEY"]

In [ ]:
data = {
    "origins": [
        {
            "waypoint": {
                "location": {
                    "latLng": {"latitude": 37.420761, "longitude": -122.081356}
                }
            },
        },
        {
            "waypoint": {
                "location": {
                    "latLng": {"latitude": 37.403184, "longitude": -122.097371}
                }
            },
        },
    ],
    "destinations": [
        {
            "waypoint": {
                "location": {
                    "latLng": {"latitude": 37.420999, "longitude": -122.086894}
                }
            }
        },
        {
            "waypoint": {
                "location": {
                    "latLng": {"latitude": 37.383047, "longitude": -122.044651}
                }
            }
        },
    ],
    "travelMode": "TRANSIT",
    # 9:00 UTC is 11:00 CEST
    "departureTime": "2023-09-04T09:00:00Z",
}

In [ ]:
import os


response = requests.post(
    "https://routes.googleapis.com/distanceMatrix/v2:computeRouteMatrix",
    json=data,
    headers={
        "X-Goog-Api-Key": GMAPS_API_KEY,
        "X-Goog-FieldMask": "originIndex,destinationIndex,duration,distanceMeters,status,condition",
    },
)

In [ ]:
# 47.378N, 8.540E

In [ ]:
# zurich = Location(47.378, 8.540)
zurich = Location(47.378, 8.540)
# zurich = Location(45, 130)

In [ ]:
zoom = 14
grid = make_grid(zurich, zoom)

image = get_static_map(zurich, zoom, markers=grid)

IPython.display.Image(image)

In [ ]:
response.content